# Object Oriented Suites

In [1]:
# Following code is needed to preconfigure this notebook
import sys
import os
sys.path.insert(0, os.path.abspath('../../..'))

import pyflow as pf

scratchdir = os.path.join('/', 'path', 'to', 'scratch')
filesdir = os.path.join(scratchdir, 'files')
outdir = os.path.join(scratchdir, 'out')

## Suite Structural Layout

As a **pyflow** user, you are encouraged to use Python `with` statement to build the structure of the suites following the graphical **ecFlow** tree. Dependencies are then added to form the _Directed Graph_ for execution.

The example below creates an initial simple suite with interdependent tasks. In software terms it is essentially an example of _procedural programming_.

In [2]:
with pf.Suite('first_suite') as s:
    
    with pf.Family('family1') as f1:
        t1 = pf.Task('t1')
        with pf.Task('t2') as t2:
            pf.Variable('FOO', 'bar')
            
        t1 >> t2
        
    with pf.Family('family2') as f2:
        t1 = pf.Task('t1')
        t2 = pf.Task('t2')
        t1 >> t2
        
    f1 >> f2

s

Suite(/first_suite)

Whilst procedural programming can be used to build simple suites, to manage long-term lifecycles of complex suites we encourage drawing inspiration from object-oriented software development.

Suites can be split into objects that are derived from **pyflow** components. Suites can then be assembled from those configurable and reusable objects.

### Deriving From Task

Probably the most important **pyflow** class to subclass is `pf.Task`. This object describes what should be carried out as one executable unit.

Consider the following _non-object-oriented_ task definition built within a **Family**.

In [3]:
with pf.Family('f') as f:
    
    variables = {
        'HALF': 7,
        'LIMIT': 2*7
    }
        
    labels = {
        'a_label': 'with a value'
    }
    
    t = pf.Task('my_task', labels=labels, defstatus=pf.state.suspended, variables=variables)
    
    # Note that t is incomplete at this point...
    t.script = [
        'echo "This is a counting task ..."',
        'for i in $(seq 1 $HALF); do echo "count $i/$LIMIT"; done',
        'i=$[$HALF+1]; while [ $i -lt $LIMIT ]; do echo "count $i/$LIMIT" ; i=$[$i+1]; done'
    ]

f

Family(/f)

As a suite grows, and the number of tasks increases, the complexity of managing all of these components becomes prohibitive.

We wish to _encapsulate_ all of the functionality related to this task into a single object. As we want to reuse functionality we organise objects into classes. These classes should be appropriately configurable.

As the number of tasks increases, we can re-use the class to create objects with similar behaviour. This in turn will dramatically reduce the complexity of the families and then of the suites.

The above task should now be defined as a reusable class.

In [4]:
class MyTask(pf.Task):
    
    """Counts to the double of a number, first half using a for loop then a while loop"""
    
    def __init__(self, name, default_value=0, **kwargs):
        
        variables = {
            'HALF': default_value,
            'LIMIT': 2*default_value,
        }
        variables.update(kwargs.pop('variables', {}))
        
        labels = {
            'counter_label': 'count to {}'.format(2*default_value)
        }
        
        script = [
            'echo "This is a counting task named {}"'.format(name),
            'for i in $(seq 1 $HALF); do echo "count $i/$LIMIT"; done',
            'i=$[$HALF+1]; while [ $i -lt $LIMIT ]; do echo "count $i/$LIMIT" ; i=$[$i+1]; done'
        ]
        
        super().__init__(name,
                         script=script,
                         labels=labels,
                         variables=variables,
                         **kwargs)


with pf.Suite('CountingSuite', files=os.path.join(filesdir, 'CountingSuite')) as s:
    with pf.Family('F') as f:
        MyTask('Seven', 7, defstatus=pf.state.suspended)
        MyTask('Five', 5)
    
s

Suite(/CountingSuite)

### Deriving from Family and other pyflow objects

The same process can be used for deriving from families or other **pyflow** related classes. In this manner we can build up configurable functionality piece by piece.

Note how the family takes an input parameter `counters`, to control how many tasks it generates internally.

In [5]:
class MyFamily(pf.Family):
    
    def __init__(self, name, counters, **kwargs):
        
        labels = {
            'total_counters': counters
        }
        
        super().__init__(name, labels=labels, **kwargs)
        
        with self:
            pf.sequence(MyTask('{}_{}'.format(name,i), i) for i in range(counters))


with pf.Suite('CountingSuite', files=os.path.join(filesdir, 'CountingSuite')) as s:
    f = MyFamily('TaskCounter', 7)
    
f

MyFamily(/CountingSuite/TaskCounter)

### Composing Suites from Reusable Components

All objects in the suite can be constructed and configured. It is worth noting that the derived class can be used within Python `with` statements in the same way as the base classes. This allows us to set some values or defaults without _forcing_ us to build the entire suite inside the constructor of a derived type.

In [6]:
class CourseSuite(pf.Suite):
    """
    This CourseSuite object will be used throughout the course to provide sensible
    defaults without verbosity
    """
    def __init__(self, name, **kwargs):
        
        config = {
            'host': pf.LocalHost(),
            'files': os.path.join(filesdir, name),
            'home': outdir,
            'defstatus': pf.state.suspended
        }
        config.update(kwargs)
        
        super().__init__(name, **config)

         
with CourseSuite('configurable_suite') as s:
    MyFamily('fam1', 3)
    MyFamily('fam2', 5)
    
s

CourseSuite(/configurable_suite)

**pyflow** aims to provide a library of commonly used abstract functionality, but suites should aim to build and collect classes of internally useful functionality which can be used to build a suite out of relevant objects.